In [1]:
import os
os.makedirs('output', exist_ok=True)

import sys
sys.path.append('../')

from utils import load_walksheds, overlay_wks

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

* wksp5, wksp75 contain the .5 and .75 walkshed boundaries, respectively
* sop contains the SoP indices and the bikelanes variable
* mstns contains the MSTN station IDs

In [2]:
wksp5, wksp75 = load_walksheds()

In [3]:
sop = gpd.read_file("../../Data/SoP_BikeLanes_Feb_2024.geojson")
mstns = pd.read_excel("../../Data/mstn_id_to_stn_name.xlsx")

* Sort and and create a new column of station names for easy merging
* Merge the walksheds and SoP
* Group by the station name and sum the bike lanes, take the mean of the SoP NORMALIZED indices
* Merge in the station IDs and keep only relevant columns
* Export

In [4]:
mstns.sort_values(by = 'PRIMARY_NAME_FY23',inplace=True)
mstns['walkshed_filename'] = ['ADDISON ROAD-SEAT PLEASANT','ANACOSTIA','ARCHIVES-NAVY MEMORIAL-PENN QUARTER','ARLINGTON CEMETERY','Ashburn','BALLSTON-MU',
                              'BENNING ROAD','BETHESDA','BRADDOCK ROAD','BRANCH AVE','BROOKLAND-CUA','CAPITOL HEIGHTS','CAPITOL SOUTH','CHEVERLY',
                              'CLARENDON','CLEVELAND PARK','COLLEGE PARK-U OF MD','COLUMBIA HEIGHTS','CONGRESS HEIGHTS','COURT HOUSE','CRYSTAL CITY',
                              'DEANWOOD','LARGO TOWN CENTER','Washington Dulles International Airport', 'DUNN LORING-MERRIFIELD','DUPONT CIRCLE','EAST FALLS CHURCH','EASTERN MARKET','EISENHOWER AVENUE','FARRAGUT NORTH',
                              'FARRAGUT WEST','FEDERAL CENTER SW','FEDERAL TRIANGLE','FOGGY BOTTOM-GWU','FOREST GLEN','FORT TOTTEN','FRANCONIA-SPRINGFIELD',
                              'FRIENDSHIP HEIGHTS','GALLERY PL-CHINATOWN','GEORGIA AVE-PETWORTH','GLENMONT','GREENBELT','GREENSBORO','GROSVENOR-STRATHMORE',
                              'Herndon','HUNTINGTON',"PRINCE GEORGE'S PLAZA",'Innovation Center','JUDICIARY SQUARE','KING ST-OLD TOWN',"L'ENFANT PLAZA",'LANDOVER',
                              'Loudoun Gateway','MCLEAN','MCPHERSON SQUARE','MEDICAL CENTER','METRO CENTER','MINNESOTA AVE','MORGAN BOULEVARD',
                              'MT VERNON SQ 7TH ST-CONVENTION CENTER','NAVY YARD-BALLPARK','NAYLOR ROAD','NEW CARROLLTON','NOMA-GALLAUDET','WHITE FLINT','PENTAGON',
                              'PENTAGON CITY','POTOMAC AVE', "",'Reston Town Center','RHODE ISLAND AVE-BRENTWOOD','ROCKVILLE','RONALD REAGAN WASHINGTON NATIONAL AIRPORT',
                              'ROSSLYN','SHADY GROVE','SHAW-HOWARD U','SILVER SPRING','SMITHSONIAN','SOUTHERN AVENUE','SPRING HILL',
                              'STADIUM-ARMORY','SUITLAND','TAKOMA','TENLEYTOWN-AU','TWINBROOK','TYSONS CORNER','U STREET/AFRICAN-AMER CIVIL WAR MEMORIAL/CARDOZO',
                              'UNION STATION','VAN DORN STREET','VAN NESS-UDC','VIENNA/FAIRFAX-GMU','VIRGINIA SQUARE-GMU','WATERFRONT','WEST FALLS CHURCH-VT/UVA',
                              'WEST HYATTSVILLE','WHEATON','WIEHLE-RESTON EAST','WOODLEY PARK-ZOO/ADAMS MORGAN',]

In [5]:
walksheds_sop50 = gpd.sjoin(wksp5, sop, how='left')
walksheds_sop75 = gpd.sjoin(wksp75, sop, how='left')

In [6]:
walksheds_sop50 = walksheds_sop50.groupby('Name_1').agg({'bikelane': 'sum', 'form3norm' : 'mean', 'dens2norm': 'mean', 'prox3norm': 'mean',
       'conn6norm': 'mean', 'parks2norm': 'mean', 'peds4norm': 'mean', 'safenorm': 'mean', 'traffic5norm': 'mean',
       'aesttot3norm': 'mean', 'paf2norm': 'mean', 'sop7norm': 'mean'})
walksheds_sop50.reset_index(inplace=True)

walksheds_sop75 = walksheds_sop75.groupby('Name_1').agg({'bikelane': 'sum', 'form3norm' : 'mean', 'dens2norm': 'mean', 'prox3norm': 'mean',
       'conn6norm': 'mean', 'parks2norm': 'mean', 'peds4norm': 'mean', 'safenorm': 'mean', 'traffic5norm': 'mean',
       'aesttot3norm': 'mean', 'paf2norm': 'mean', 'sop7norm': 'mean'})
walksheds_sop75.reset_index(inplace=True)

In [7]:
walksheds_sop50_2 = mstns.merge(walksheds_sop50, left_on = 'walkshed_filename', right_on= 'Name_1')
walksheds_sop50_2 = walksheds_sop50_2[['ID', 'Name_1', 'bikelane', 'form3norm',
       'dens2norm', 'prox3norm', 'conn6norm', 'parks2norm', 'peds4norm',
       'safenorm', 'traffic5norm', 'aesttot3norm', 'paf2norm', 'sop7norm']]

walksheds_sop75_2 = mstns.merge(walksheds_sop75, left_on = 'walkshed_filename', right_on= 'Name_1')
walksheds_sop75_2 = walksheds_sop75_2[['ID', 'Name_1', 'bikelane', 'form3norm',
       'dens2norm', 'prox3norm', 'conn6norm', 'parks2norm', 'peds4norm',
       'safenorm', 'traffic5norm', 'aesttot3norm', 'paf2norm', 'sop7norm']]



In [13]:
walksheds_sop50_2.to_csv('output/SoP_BikeLanes_p5.csv')
walksheds_sop75_2.to_csv('output/SoP_BikeLanes_p75.csv')